In [62]:
from matching.games import HospitalResident
import random;

def del_player(p, choices, hospital_preferences):
    del choices[p]
    for (h, pl) in hospital_preferences.items():
        if p in pl:
            pl.remove(p)

def del_hospital(h, choices, hospital_preferences, capacities):
    del hospital_preferences[h]
    del capacities[h]
    for (p, hl) in choices.items():
        if h in hl:
            hl.remove(h)
def cleanup(choices, hospital_preferences, capacities):
    #cleanup : delete hospital and players that are finished
        cleanup = True
        while cleanup: 
            cleanup = False
            to_remove = []
            for (h, p) in hospital_preferences.items():
                #if an hospital is full or noone wants to go there, delete it
                if not (capacities[h] > 0 and len(p) > 0):
                    to_remove.append(h)
                    cleanup = True
            for h in to_remove:
                del_hospital(h, choices, hospital_preferences, capacities)
            
            #if a player don't want anything anymore, delete it
            to_remove = []
            for (p, hl) in choices.items():
                if len(hl) == 0:
                    to_remove.append(p)
                    cleanup = True    
            for p in to_remove:
                del_player(p, choices, hospital_preferences)  

def match(choices, capacities):
    players = []
    hospital_preferences = dict()
    assignments = dict()
    #fill the assignments and hospital pref with empty lists
    for (h, c) in capacities.items():
        assignments[h] = []
        hospital_preferences[h] = []
    #add all players that wanted an hospital to that hospital's preference list
    for (p, c) in choices.items():
        for h in c:
            hospital_preferences[h].append(p)
    #shuffle the hospital preference list
    for (h, p) in hospital_preferences.items():
            random.shuffle(p)
            c = 0
            for i in range(len(p) - 1, -1, -1):
                while c < i and len(choices[p[c]]) == 1:
                    c += 1
                if c >= i:
                        break
                if len(choices[p[i]]) == 1:
                    p[c], p[i] = p[i], p[c]
                    c += 1
            print(p)
    #First round to attribute all the resident who have only one choice
    cont = True
    while cont:
        cont = False
        P1C = []
        for (p, r) in choices.items():
            if len(r) == 1:
                cont = True
                P1C.append((p, r[0]))
        if not cont:
            break
        random.shuffle(P1C)
        for (p, h) in P1C:
            if capacities[h] == 0:
                del_player(p, choices, hospital_preferences)
                continue
            capacities[h] -= 1
            assignments[h].append(p)
            del_player(p, choices, hospital_preferences)
        cleanup(choices, hospital_preferences, capacities)
    #Delete the hospital that are full, and restart while there are some residents with only one choice.

    #while there are still things waiting to be assigned
    cont = True
    while cont:
        cont = False
        #create a matching
        game = HospitalResident.create_from_dictionaries(choices, hospital_preferences, capacities)
        matching = game.solve(optimal="resident")
        #process it
        for (h, ass) in matching.items():
            h = str(h)
            #add it to the assignments
            assignments[h] += ass
            #substract the capacities
            capacities[h] -= len(ass)
            #prevent a player from getting assigned to the same hospital twice
            for p in ass:
                p = str(p)
                hospital_preferences[h].remove(p)
                choices[p].remove(h)
        #shuffle the hospital preferences and see if it should continue
        for (h, p) in hospital_preferences.items():
            random.shuffle(hospital_preferences[h])
            if capacities[h] > 0 and len(p) > 0:
                cont = True
        cleanup(choices, hospital_preferences, capacities)
    return assignments

In [97]:
choices = {"j1":["a1", "a2", "a3"], "j2":["a2","a3"], "j3":["a1"]}
cap = {"a1":1,"a2":1, "a3":10}
print(match(choices, cap))

['j3', 'j1']
['j1', 'j2']
['j1', 'j2']
{'a1': ['j3'], 'a2': [j1], 'a3': [j2, j1]}
